try again

In [29]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
dtype=torch.FloatTensor #32位的float

preprocess data

In [36]:
sentences=['i like cat','i hate coffee','i enjoy everyday','i love you']
sentences_list=" ".join(sentences).split()
vocab=list(set(sentences_list))
word2idx={w:i for i,w in enumerate(vocab)}
idx2word={i:w for i,w in enumerate(vocab)}
V=len(vocab)
print(V)
m=2
n_step=2
#选x,y
def make_data(sentences):
    x=[]
    y=[]
    for sen in sentences:
        sen=sen.split()
        x.append([word2idx[word] for word in sen[:-1]])
        y.append(word2idx[sen[-1]])
    return x,y

dataset=make_data(sentences)
x,y=dataset
x,y=torch.LongTensor(x),torch.LongTensor(y)
print(x,y)
dataset=TensorDataset(x,y)
dataloader=DataLoader(dataset,2,True)

9
tensor([[2, 1],
        [2, 0],
        [2, 8],
        [2, 5]]) tensor([6, 4, 3, 7])


model

In [37]:
n_hidden=10
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM,self).__init__()
        self.C=nn.Embedding(V,m)
        self.H=nn.Parameter(torch.randn(n_step*m,n_hidden).type(dtype))
        self.d=nn.Parameter(torch.randn(n_hidden).type(dtype))
        
        self.U=nn.Parameter(torch.randn(n_hidden,V).type(dtype))
        self.b=nn.Parameter(torch.randn(V).type(dtype))
        self.W=nn.Parameter(torch.randn(n_step*m,V).type(dtype))
    def forward(self,X):
        X=self.C(X)
        X=X.view(-1,n_step*m)
        hidden_out=torch.tanh(self.d+torch.mm(X,self.H))
        y=self.b+torch.mm(X,self.W)+torch.mm(hidden_out,self.U)
        return y
model=NNLM()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)
criterion=nn.CrossEntropyLoss()

train

In [38]:
writer=SummaryWriter()
step=0

for epoch in range(500):
    loss_record=[]
    for batch_x,batch_y in tqdm(dataloader):

        pred=model(batch_x)
        loss=criterion(pred,batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        
        step+=1
        loss_record.append(loss.item())
    mean_train_loss=sum(loss_record)/len(loss_record)
    writer.add_scalar("TrainLoss",mean_train_loss,step)

100%|██████████| 2/2 [00:00<00:00, 500.57it/s]


In [46]:
pred=model(x).max(1)[-1]
idx=[idx.item() for idx in pred]
print([idx2word[i] for i in idx])

['cat', 'coffee', 'everyday', 'you']
